In [129]:
import os
import json
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt
from panel.interact import interact
import panel as pn
pn.extension('plotly')

In [130]:
load_dotenv()

True

In [131]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [132]:
start_date = pd.Timestamp("2015-01-15", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-01-15", tz="America/New_York").isoformat()
tickers = ["PFE", "MRNA", "REGN", "MSFT", "SNE"]

# Create the shares DataFrame
timeframe = "1D"

stock_tickers = api.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

stock_tickerspx = pd.DataFrame()
stock_tickerspx["PFE"] = stock_tickers["PFE"]["close"]
stock_tickerspx["MRNA"] = stock_tickers["MRNA"]["close"]
stock_tickerspx["REGN"] = stock_tickers["REGN"]["close"]
stock_tickerspx["MSFT"] = stock_tickers["MSFT"]["close"]
stock_tickerspx["SNE"] = stock_tickers["SNE"]["close"]

# Drop the time component of the date
stock_tickerspx.index = stock_tickers.index.date
index = stock_tickerspx.index
index.name = "Date"
stock_tickerspx.reset_index(level=0, inplace=True)

# Display sample data
stock_tickerspx.head()


,Date,PFE,MRNA,REGN,MSFT,SNE
0,2015-01-15,32.40,0.7600,397.13,45.48,20.94
1,2015-01-16,32.78,0.7900,408.28,46.22,20.68
2,2015-01-20,33.10,0.7650,421.11,46.39,20.83
3,2015-01-21,32.66,0.7400,418.11,45.91,21.91
4,2015-01-22,32.80,0.7299,420.82,47.12,22.17


In [133]:
# Creating signals for Pfizer

# Grab just the `date` and `close` from the dataset
signals_df1 = stock_tickerspx.loc[:,["Date", "PFE"]].copy()

# Set the short window and long windows
short_window = 20
long_window = 100

# Set the `date` column as the index
signals_df1 = signals_df1.set_index("Date", drop=True)

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df1["SMA20"] = signals_df1.iloc[80:]["PFE"].rolling(window=short_window).mean()
signals_df1["SMA100"] = signals_df1["PFE"].rolling(window=long_window).mean()
signals_df1["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA20 is under the SMA100, and
# where 1 is when the SMA20 is higher (or crosses over) the SMA100
signals_df1["Signal"][short_window:] = np.where(
    signals_df1["SMA20"][short_window:] > signals_df1["SMA100"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df1["Entry/Exit"] = signals_df1["Signal"].diff()

signals_df1.head(10)

,PFE,SMA20,SMA100,Signal,Entry/Exit
Date,,,,,
2015-01-15,32.40,NaN,NaN,0.0,NaN
2015-01-16,32.78,NaN,NaN,0.0,0.0
2015-01-20,33.10,NaN,NaN,0.0,0.0
2015-01-21,32.66,NaN,NaN,0.0,0.0
2015-01-22,32.80,NaN,NaN,0.0,0.0
2015-01-23,32.45,NaN,NaN,0.0,0.0
2015-01-26,32.79,NaN,NaN,0.0,0.0
2015-01-27,32.63,NaN,NaN,0.0,0.0
2015-01-28,31.96,NaN,NaN,0.0,0.0


In [134]:
#plotting Pfizer signals
def pfizer_plot():
# Visualize exit position relative to close price
    exit = signals_df1[signals_df1['Entry/Exit'] == -1.0]['PFE'].hvplot.scatter(
        color='red',
        marker='v',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Visualize entry position relative to close price
    entry = signals_df1[signals_df1['Entry/Exit'] == 1.0]['PFE'].hvplot.scatter(
        color='green',
        marker='^',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Visualize close price for the investment
    security_close = signals_df1[['PFE']].hvplot(
        line_color='lightgray',
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Visualize moving averages
    moving_avgs = signals_df1[['SMA20', 'SMA100']].hvplot(
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Overlay plots
    entry_exit_plot = security_close * moving_avgs * entry * exit
    entry_exit_plot.opts(xaxis=None, title = "Pfizer")
    
    return entry_exit_plot

In [135]:
# Creating signals for Moderna

# Grab just the `date` and `close` from the dataset
signals_df2 = stock_tickerspx.loc[:,["Date", "MRNA"]].copy()

# Set the short window and long windows
short_window = 20
long_window = 100

# Set the `date` column as the index
signals_df2 = signals_df2.set_index("Date", drop=True)

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df2["SMA20"] = signals_df2.iloc[80:]["MRNA"].rolling(window=short_window).mean()
signals_df2["SMA100"] = signals_df2["MRNA"].rolling(window=long_window).mean()
signals_df2["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA20 is under the SMA100, and
# where 1 is when the SMA20 is higher (or crosses over) the SMA100
signals_df2["Signal"][short_window:] = np.where(
    signals_df2["SMA20"][short_window:] > signals_df2["SMA100"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df2["Entry/Exit"] = signals_df2["Signal"].diff()

signals_df2.head(10)

,MRNA,SMA20,SMA100,Signal,Entry/Exit
Date,,,,,
2015-01-15,0.7600,NaN,NaN,0.0,NaN
2015-01-16,0.7900,NaN,NaN,0.0,0.0
2015-01-20,0.7650,NaN,NaN,0.0,0.0
2015-01-21,0.7400,NaN,NaN,0.0,0.0
2015-01-22,0.7299,NaN,NaN,0.0,0.0
2015-01-23,0.6850,NaN,NaN,0.0,0.0
2015-01-26,0.6820,NaN,NaN,0.0,0.0
2015-01-27,0.6648,NaN,NaN,0.0,0.0
2015-01-28,0.6200,NaN,NaN,0.0,0.0


In [136]:
#plotting Moderna signals
def moderna_plot():
# Visualize exit position relative to close price
    exit = signals_df2[signals_df2['Entry/Exit'] == -1.0]['MRNA'].hvplot.scatter(
        color='red',
        marker='v',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Visualize entry position relative to close price
    entry = signals_df2[signals_df2['Entry/Exit'] == 1.0]['MRNA'].hvplot.scatter(
        color='green',
        marker='^',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Visualize close price for the investment
    security_close = signals_df2[['MRNA']].hvplot(
        line_color='lightgray',
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Visualize moving averages
    moving_avgs = signals_df2[['SMA20', 'SMA100']].hvplot(
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Overlay plots
    entry_exit_plot2 = security_close * moving_avgs * entry * exit
    entry_exit_plot2.opts(xaxis=None, title = "Moderna")
    
    return entry_exit_plot2

In [137]:
# Creating signals for Regeneron

# Grab just the `date` and `close` from the dataset
signals_df3 = stock_tickerspx.loc[:,["Date", "REGN"]].copy()

# Set the short window and long windows
short_window = 20
long_window = 100

# Set the `date` column as the index
signals_df3 = signals_df3.set_index("Date", drop=True)

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df3["SMA20"] = signals_df3.iloc[80:]["REGN"].rolling(window=short_window).mean()
signals_df3["SMA100"] = signals_df3["REGN"].rolling(window=long_window).mean()
signals_df3["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA20 is under the SMA100, and
# where 1 is when the SMA20 is higher (or crosses over) the SMA100
signals_df3["Signal"][short_window:] = np.where(
    signals_df3["SMA20"][short_window:] > signals_df3["SMA100"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df3["Entry/Exit"] = signals_df3["Signal"].diff()

signals_df3.tail(10)

,REGN,SMA20,SMA100,Signal,Entry/Exit
Date,,,,,
2021-01-04,482.46,489.1770,555.628829,0.0,0.0
2021-01-05,483.01,488.7050,554.335429,0.0,0.0
2021-01-06,469.65,487.3965,552.923229,0.0,0.0
2021-01-07,481.20,486.5975,551.610329,0.0,0.0
2021-01-08,498.81,487.1260,550.386329,0.0,0.0
2021-01-11,502.46,488.2750,549.225029,0.0,0.0
2021-01-12,505.75,489.5935,547.983929,0.0,0.0
2021-01-13,512.19,490.2030,546.936399,0.0,0.0
2021-01-14,517.90,491.4885,546.061499,0.0,0.0


In [138]:
#plotting Regeneron signals
def regeneron_plot():
# Visualize exit position relative to close price
    exit = signals_df3[signals_df3['Entry/Exit'] == -1.0]['REGN'].hvplot.scatter(
        color='red',
        marker='v',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Visualize entry position relative to close price
    entry = signals_df3[signals_df3['Entry/Exit'] == 1.0]['REGN'].hvplot.scatter(
        color='green',
        marker='^',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Visualize close price for the investment
    security_close = signals_df3[['REGN']].hvplot(
        line_color='lightgray',
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Visualize moving averages
    moving_avgs = signals_df3[['SMA20', 'SMA100']].hvplot(
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Overlay plots
    entry_exit_plot3 = security_close * moving_avgs * entry * exit
    entry_exit_plot3.opts(xaxis=None, title = "Regeneron")
    
    return entry_exit_plot3

In [139]:
# Creating signals for Microsoft

# Grab just the `date` and `close` from the dataset
signals_df4 = stock_tickerspx.loc[:,["Date", "MSFT"]].copy()

# Set the short window and long windows
short_window = 20
long_window = 100

# Set the `date` column as the index
signals_df4 = signals_df4.set_index("Date", drop=True)

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df4["SMA20"] = signals_df4.iloc[80:]["MSFT"].rolling(window=short_window).mean()
signals_df4["SMA100"] = signals_df4["MSFT"].rolling(window=long_window).mean()
signals_df4["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA20 is under the SMA100, and
# where 1 is when the SMA20 is higher (or crosses over) the SMA100
signals_df4["Signal"][short_window:] = np.where(
    signals_df4["SMA20"][short_window:] > signals_df4["SMA100"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df4["Entry/Exit"] = signals_df4["Signal"].diff()

signals_df4.tail(10)

,MSFT,SMA20,SMA100,Signal,Entry/Exit
Date,,,,,
2021-01-04,217.81,218.31975,213.809356,1.0,0.0
2021-01-05,218.01,218.50025,213.896556,1.0,0.0
2021-01-06,212.22,218.39525,213.931556,1.0,0.0
2021-01-07,218.29,218.51025,214.025556,1.0,0.0
2021-01-08,219.55,218.91725,214.118564,1.0,0.0
2021-01-11,217.50,219.26650,214.178764,1.0,0.0
2021-01-12,214.93,219.34950,214.231064,1.0,0.0
2021-01-13,216.41,219.46050,214.249664,1.0,0.0
2021-01-14,213.02,219.40350,214.249664,1.0,0.0


In [140]:
#plotting Microsoft signals
def microsoft_plot():
# Visualize exit position relative to close price
    exit = signals_df4[signals_df4['Entry/Exit'] == -1.0]['MSFT'].hvplot.scatter(
        color='red',
        marker='v',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Visualize entry position relative to close price
    entry = signals_df4[signals_df4['Entry/Exit'] == 1.0]['MSFT'].hvplot.scatter(
        color='green',
        marker='^',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Visualize close price for the investment
    security_close = signals_df4[['MSFT']].hvplot(
        line_color='lightgray',
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Visualize moving averages
    moving_avgs = signals_df4[['SMA20', 'SMA100']].hvplot(
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Overlay plots
    entry_exit_plot4 = security_close * moving_avgs * entry * exit
    entry_exit_plot4.opts(xaxis=None, title = "Microsoft")

    return entry_exit_plot4

In [141]:
# Creating signals for Sony

# Grab just the `date` and `close` from the dataset
signals_df5 = stock_tickerspx.loc[:,["Date", "SNE"]].copy()

# Set the short window and long windows
short_window = 20
long_window = 100

# Set the `date` column as the index
signals_df5 = signals_df5.set_index("Date", drop=True)

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df5["SMA20"] = signals_df5.iloc[80:]["SNE"].rolling(window=short_window).mean()
signals_df5["SMA100"] = signals_df5["SNE"].rolling(window=long_window).mean()
signals_df5["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA20 is under the SMA100, and
# where 1 is when the SMA20 is higher (or crosses over) the SMA100
signals_df5["Signal"][short_window:] = np.where(
    signals_df5["SMA20"][short_window:] > signals_df5["SMA100"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df5["Entry/Exit"] = signals_df5["Signal"].diff()

signals_df5.tail(10)

,SNE,SMA20,SMA100,Signal,Entry/Exit
Date,,,,,
2021-01-04,100.07,96.7085,84.259299,1.0,0.0
2021-01-05,103.09,97.1430,84.482899,1.0,0.0
2021-01-06,101.08,97.5290,84.666999,1.0,0.0
2021-01-07,102.00,97.9610,84.857699,1.0,0.0
2021-01-08,103.99,98.4520,85.063099,1.0,0.0
2021-01-11,103.84,98.9395,85.271599,1.0,0.0
2021-01-12,104.04,99.4080,85.500999,1.0,0.0
2021-01-13,103.89,99.9335,85.745799,1.0,0.0
2021-01-14,103.42,100.3980,85.992499,1.0,0.0


In [142]:
#plotting Sony signals
def sony_plot():
# Visualize exit position relative to close price
    exit = signals_df5[signals_df5['Entry/Exit'] == -1.0]['SNE'].hvplot.scatter(
        color='red',
        marker='v',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Visualize entry position relative to close price
    entry = signals_df5[signals_df5['Entry/Exit'] == 1.0]['SNE'].hvplot.scatter(
        color='green',
        marker='^',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Visualize close price for the investment
    security_close = signals_df5[['SNE']].hvplot(
        line_color='lightgray',
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Visualize moving averages
    moving_avgs = signals_df5[['SMA20', 'SMA100']].hvplot(
        ylabel='Price in $',
        width=1000,
        height=400
    )

# Overlay plots
    entry_exit_plot5 = security_close * moving_avgs * entry * exit
    entry_exit_plot5.opts(xaxis=None, title = "Sony")
    
    return entry_exit_plot5

In [143]:
pharma_plot = pn.Column(
    "## Pharmacuetical Entry / Exit points.",
    pfizer_plot(),
    moderna_plot(),
    regeneron_plot()
)

tech_plot = pn.Column(
    "## Technology Entry / Exit points.",
    microsoft_plot(),
    sony_plot(),
)

Entry_exit_dashboard = pn.Tabs(
    ("Pharmacueticals Exit / Entry points", pharma_plot),
    ("Technology Entry / Exit points", tech_plot)
)

Entry_exit_dashboard

Tabs
    [0] Column
        [0] Markdown(str)
        [1] HoloViews(Overlay)
        [2] HoloViews(Overlay)
        [3] HoloViews(Overlay)
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Overlay)
        [2] HoloViews(Overlay)